In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.5
pandas 0.25.2
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
# 读取数据
train_file = './data/train.csv'
eval_file = './data/eval.csv'

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

print(train_df.head())
print(eval_df.head())

   survived     sex   age  n_siblings_spouses  parch     fare  class     deck  \
0         0    male  22.0                   1      0   7.2500  Third  unknown   
1         1  female  38.0                   1      0  71.2833  First        C   
2         1  female  26.0                   0      0   7.9250  Third  unknown   
3         1  female  35.0                   1      0  53.1000  First        C   
4         0    male  28.0                   0      0   8.4583  Third  unknown   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
2  Southampton     y  
3  Southampton     n  
4   Queenstown     y  
   survived     sex   age  n_siblings_spouses  parch     fare   class  \
0         0    male  35.0                   0      0   8.0500   Third   
1         0    male  54.0                   0      0  51.8625   First   
2         1  female  58.0                   0      0  26.5500   First   
3         1  female  55.0                   0      0  16.0000  Second   
4         

In [3]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(eval_df.head())
print(y_train.head())
print(y_eval.head())


      sex   age  n_siblings_spouses  parch     fare  class     deck  \
0    male  22.0                   1      0   7.2500  Third  unknown   
1  female  38.0                   1      0  71.2833  First        C   
2  female  26.0                   0      0   7.9250  Third  unknown   
3  female  35.0                   1      0  53.1000  First        C   
4    male  28.0                   0      0   8.4583  Third  unknown   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
2  Southampton     y  
3  Southampton     n  
4   Queenstown     y  
      sex   age  n_siblings_spouses  parch     fare   class     deck  \
0    male  35.0                   0      0   8.0500   Third  unknown   
1    male  54.0                   0      0  51.8625   First        E   
2  female  58.0                   0      0  26.5500   First        C   
3  female  55.0                   0      0  16.0000  Second  unknown   
4    male  34.0                   0      0  13.0000  Second        D   

  

In [4]:
# 离散特征
categorical_colums = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
numerical_columns = ['age','fare']

feature_columns = []
# 离散
for categorical_column in categorical_colums:
    vocab = train_df[categorical_column].unique() # 获取所有可能的值
    print(categorical_column,vocab)
    # one-hot 编码，
    feature_columns.append(
        tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(categorical_column,vocab)))

# 连续
for numerical_column in numerical_columns:
    feature_columns.append(tf.feature_column.numeric_column(numerical_column,dtype=tf.float32))
    
# cross feature 交叉特征
feature_columns.append(tf.feature_column.indicator_column(tf.feature_column.crossed_column(['age','sex'],hash_bucket_size=100)))

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [5]:
# 构建dataset
def make_dataset(data_df,label_df,epochs=10,shuffle=True,batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [6]:
# 保存输出模型
output_dir = 'baseline_model_new_features'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

baseline_estimator = tf.estimator.BaselineClassifier(model_dir = output_dir,n_classes=2)
baseline_estimator.train(input_fn = lambda:make_dataset(train_df,y_train,epochs=100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'baseline_model_new_features', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000014E4E43CDC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at F:\public\anconda\envs\tf20\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]

Original stack trace for 'IteratorGetNext':
  File "F:\public\anconda\envs\tf20\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "F:\public\anconda\envs\tf20\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "F:\public\anconda\envs\tf20\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "F:\public\anconda\envs\tf20\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "F:\public\anconda\envs\tf20\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "F:\public\anconda\envs\tf20\

In [ ]:
baseline_estimator.evaluate(input_fn = lambda : make_dataset(eval_df,y_eval,epochs=1,shuffle=False,batch_size=20))

In [ ]:
linear_ouput_dir = 'linear_model_new_features'
if not os.path.exists(linear_ouput_dir):
    os.mkdir(linear_ouput_dir)
linear_estimator = tf.estimator.LinearClassifier(model_dir=linear_ouput_dir,n_classes=2,feature_columns=feature_columns)
linear_estimator.train(input_fn=lambda :make_dataset(train_df,y_train,epochs=100))

In [ ]:
linear_estimator.evaluate(input_fn=lambda :make_dataset(eval_df,y_eval,epochs=1,shuffle=False))

In [ ]:
dnn_output_dir = './dnn_model_new_features'
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
dnn_estimator = tf.estimator.DNNClassifier(model_dir=dnn_output_dir,n_classes=2,feature_columns=feature_columns,
                                          hidden_units=[128,128],activation_fn=tf.nn.relu,optimizer='Adam')
dnn_estimator.train(input_fn=lambda : make_dataset(train_df,y_train,epochs=100))

In [ ]:
dnn_estimator.evaluate(input_fn=lambda : make_dataset(eval_df,y_eval,epochs=1,shuffle=False))